# Recall and MRR Score Calculation

In [47]:
# Import necessary libraries
import pandas as pd
import numpy as np

In [48]:
#importing the query relevance file
df_qrel = pd.read_csv('antique_qurel_test.csv')
df_qrel = df_qrel[['query_id','doc_id','relevance']]

In [49]:
# Define a function to map relevance values to binary labels
def map_values(value):
    if value in [3, 4]:
        return 1
    else:
        return 0

In [ ]:
# Apply the mapping function to create binary labels and add to the relevance data
df_qrel['label'] = df_qrel['relevance'].apply(map_values)

# Drop duplicate entries based on query_id and doc_id in the relevance data
df_qrel = df_qrel.drop_duplicates(subset=['query_id','doc_id'])

## Recall Calculation

In [ ]:
df_qrel = df_qrel[['query_id','doc_id','label']]

score_df = pd.read_csv('e5_base_v2_scores.csv')
score_df = score_df[['query_id','doc_id']]

In [ ]:
def calculate_average_recall(score_df, df_qrel):
    # Drop duplicates based on 'query_id' and 'doc_id'
    df = score_df.drop_duplicates(subset=['query_id', 'doc_id'])

    # Merge with df_qrel to get matching documents between predicted and qrel file
    common_ids_df = pd.merge(df, df_qrel, on=['query_id', 'doc_id'], how='inner')

    # Count occurrences of matching documents between predicted and qrel file
    query_id_counts = common_ids_df.groupby('query_id')['query_id'].count().reset_index(name='row_count')

    # Count occurrences of doc_id for each query_id in df_qrel
    doc_count_per_query = df_qrel.groupby('query_id')['doc_id'].count().reset_index(name='doc_count')

    # Merge the two count DataFrames
    recall_df = pd.merge(query_id_counts, doc_count_per_query, on=['query_id'], how='inner')

    # Calculate recall for each query_id
    recall_df['recall'] = recall_df['row_count'] / recall_df['doc_count']

    # Calculate average recall
    average_recall = sum(recall_df['recall']) / len(df_qrel['query_id'].unique())

    return average_recall



In [ ]:
avg_recall_result = calculate_average_recall(score_df, df_qrel)
print("Average Recall =", avg_recall_result)

Average Recall = 0.7510839138058918


## MRR Calculation

In [52]:
cross_enc_score_df = pd.read_csv('Cross_encoder_e5_base_v2_scores.csv')
cross_enc_score_df = cross_enc_score_df[['query_id','doc_id']]

In [60]:

def calculate_mrr(dataframe, df_qrel):
    # Filtering rows with label values as 1
    filtered_df = df_qrel[df_qrel['label'] == 1]

    # Creating a new 'rank' column based on 'query_id'
    dataframe['rank'] = dataframe.groupby('query_id').cumcount() + 1
    
    # Merging with filtered_df and filling NaNs with 0
    labeled_search_results = dataframe.merge(filtered_df, how='left', on=['query_id', 'doc_id']).fillna(0)
    
    # Grouping by query_id and label to find the minimum rank for each relevance level
    relevances_rank = labeled_search_results.groupby(['query_id', 'label'])['rank'].min()
    
    # Extracting ranks where label is 1 (relevant)
    ranks = relevances_rank.loc[:, 1]
    
    # Calculating reciprocal ranks
    reciprocal_ranks = 1 / ranks
    
    # Calculating MRR
    mrr = sum(reciprocal_ranks) / len(dataframe['query_id'].unique())
    
    return mrr


In [61]:
mrr_result = calculate_mrr(cross_enc_score_df, df_qrel)
print("MRR =", mrr_result)

MRR = 0.8654285714285714
